## Keyword find

Preproceso para unir, todos los textos de un speaker

In [ ]:
# OS BASED PATH
# base = Path(__file__).resolve().parent
# Notebook Path
import os
from glob import glob

# All files and directories ending with .txt and that don't begin with a dot:
base = os.path.abspath('')
file_paths = glob(f"{base}/*.txt")
input_files = [os.path.basename(file_path) for file_path in file_paths]
print(input_files)

nombre_salida = "transcribe"

['transcribe_4339_6.txt', 'transcribe_8701_2.txt', 'transcribe_9230_3.txt', 'transcribe_4369_1.txt']


In [ ]:
import re

speakers1_times = {}
speakers2_times = {}
speakers1_dict = {}
speakers2_dict = {}
for numero_diar, input_file in enumerate(input_files):
  with open(input_file, "r", encoding="UTF-8") as f:
      transcribe = f.readlines()

  # Extract the last 6 digits from the file name
  if len(input_file.split('.')) > 2:
    id_file = input_file.split('.')[1]
  else:
    id_file = input_file.split('.')[-2][-6:]

  # Keyword preprocessing
  speaker1 = [line for line in transcribe if "SPEAKER_00" in line]
  speaker2 = [line for line in transcribe if "SPEAKER_01" in line]

  # Join the lines of each speaker
  speaker1 = " ".join(speaker1)
  speaker2 = " ".join(speaker2)

  # Save the timestamps of talking of each speaker
  pattern = r"\d{1,2}:\d{2}:\d{2} - \d{1,2}:\d{2}:\d{2} \[SPEAKER_00]:"
  speaker1_tmp = re.findall(pattern, speaker1)
  speaker1_tmp = [timestamp.replace(" [SPEAKER_00]:", "") for timestamp in speaker1_timestamps]
  speakers1_times[id_file] = speaker1_tmp

  pattern = r"\d{1,2}:\d{2}:\d{2} - \d{1,2}:\d{2}:\d{2} \[SPEAKER_01]:"
  speaker2_tmp = re.findall(pattern, speaker2)
  speaker2_tmp = [timestamp.replace(" [SPEAKER_01]:", "") for timestamp in speaker2_timestamps]
  speakers2_times[id_file] = speaker2_tmp

  # Remove the strings that match the pattern 00:00:00 - 0:00:26 [SPEAKER_00]:
  pattern = r"\d{1,2}:\d{2}:\d{2} - \d{1,2}:\d{2}:\d{2} \[SPEAKER_\d{2}\]:"
  # Save the lines without timestamp
  speaker1 = re.sub(pattern, "", speaker1)
  speaker2 = re.sub(pattern, "", speaker2)

  speaker1 = speaker1.split("\n")
  speaker2 = speaker2.split("\n")
  speakers1_dict[id_file] = speaker1
  speakers2_dict[id_file] = speaker2


In [ ]:
print(speakers1_dict)
print(speakers2_dict)

{'4339_6': [' Bien, gracias a Dios. Qué bueno. Muy bien, muchas gracias. Me alegro que estés muy bien.', '  En fin, ¿qué es la reducción? Probar el cliente que, por ejemplo, yo golpeé el carro, no tengo protección.', '  Yo recorro en un carro, no que se robe, sino que un día se va a caer por ahí y eso no vale.', '  A mí me divertí.', '  Pues en ese caso tendrías cobertura en daños a terceros.', '  No, por eso te escribí. No, yo te he enviado la cotización. Tú la tienes en el WhatsApp.', '  ¿Tienes alguna cosa con ustedes? Ninguna.', '  Eso es teórico. Porque ahora mismo, ahora mismo, yo estoy con el mío. Si me meto en una peyota, se va a decir 500, 500, 500 pesos. Seguro que me llevo a hacer el tránsito y acaso me doleré por la barra. Es verdad, esto es un accidente de tránsito.', '  Y en Colombia, ni en Bogotá, ni en Río de la Mar, el árbitro.', '  Sí, tienes razón, tienes razón, ese valor, sí, a veces es muy alto.', '  Sí, sinceramente.', '  Perfectamente, te entiendo perfectamente.'

### Keyword find per item

Load spanish pipeline

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!python -m spacy download es_core_news_lg
# !python -m spacy download es_core_news_md

In [ ]:
import spacy
from spacy import displacy
import spacy

def process_speaker_data(speaker_data):
    """
    Process speaker data using spaCy.

    Args:
    - speaker_data: Dictionary containing the speaker's lines as values.

    Returns:
    - entities: List of entity labels found in the speaker's lines.
    """
    result_dict = {}

    spanish_nlp = spacy.load("es_core_news_lg")
    for key, value in speaker_data.items():
        lines = [spanish_nlp(line) for line in speaker_data[key]]
        result_dict[key] = [ent.label_ for line in lines for ent in line.ents]
    # lines = [spanish_nlp(line) for line in speaker_data.values()]
    # entities = [ent.label_ for line in lines for ent in line.ents]
    return result_dict

speaker1_entities = process_speaker_data(speakers1_dict)
speaker2_entities = process_speaker_data(speakers2_dict)


# spanish_nlp = spacy.load("es_core_news_lg")
# nlp = spacy.blank("es", vocab=spanish_nlp.vocab)

# #use spacy in each item of the speaker list
# # speaker1 = [nlp(line) for line in speaker1]
# # speaker2 = [nlp(line) for line in speaker2]
# speaker1 = [spanish_nlp(line) for line in speaker1]
# speaker2 = [spanish_nlp(line) for line in speaker2]

# # Find type of conversation using spacy
# # Find the most common entity in the conversation
# speaker1_entities = [ent.label_ for line in speaker1 for ent in line.ents]
# speaker2_entities = [ent.label_ for line in speaker2 for ent in line.ents]

print(speaker1_entities)
print(speaker2_entities)

{'4339_6': ['MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'LOC', 'LOC', 'LOC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC'], '8701_2': ['PER', 'PER', 'PER', 'PER', 'MISC', 'LOC', 'MISC', 'MISC', 'MISC', 'MISC', 'PER'], '9230_3': ['PER', 'LOC', 'MISC', 'PER', 'MISC', 'MISC', 'PER', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'LOC', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'LOC', 'ORG', 'MISC', 'LOC', 'MISC', 'MISC', 'ORG', 'LOC', 'MISC', 'ORG', 'LOC', 'ORG', 'LOC', 'LOC', 'MISC', 'MISC', 'LOC', 'MISC', 'MISC', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'LOC', 'PER', 'MISC', 'MISC', 'MISC', 'LOC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'ORG', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'ORG', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'PER', 

In [ ]:
# displacy.serve(speaker1[5], style="ent")
displacy.render(speaker1[5], style="ent", jupyter=True)

## SPACY NER Training

spaCy also supports deep learning workflows that allow connecting statistical models trained by popular machine learning libraries like Tensor Flow, PyTorch, or MXNet through its machine learning library Thinc.

In [ ]:
!pip install plac

In [ ]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

Create train set

### Entity labels and **definition**

* PERSON:      People, including fictional.
* NORP:        Nationalities or religious or political groups.
* FAC:         Buildings, airports, highways, bridges, etc.
* ORG:         Companies, agencies, institutions, etc.
* GPE:         Countries, cities, states.
* LOC:         Non-GPE locations, mountain ranges, bodies of water.
* PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
* EVENT:       Named hurricanes, battles, wars, sports events, etc.
* WORK_OF_ART: Titles of books, songs, etc.
* LAW:         Named documents made into laws.
* LANGUAGE:    Any named language.
* DATE:        Absolute or relative dates or periods.
* TIME:        Times smaller than a day.
* PERCENT:     Percentage, including ”%“.
* MONEY:       Monetary values, including unit.
* QUANTITY:    Measurements, as of weight or distance.
* ORDINAL:     “first”, “second”, etc.
* CARDINAL:    Numerals that do not fall under another type.

# Indexes finding

In [ ]:
import re

text = 'Exacto, pero si llegas a tener algún tipo de accidente, la aseguradora te va. Nosotros te prestamos el servicio.'
substring = 'aseguradora.'
type_entity = 'PRODUCT'

matches = [(match.start(), match.end()) for match in re.finditer(substring, text)]

if matches:
    start_index, end_index = matches[0]
    print(f"[({start_index}, {end_index}, '{type_entity}')]")

else:
    print("Substring not found")

[(59, 71, 'PRODUCT')]


In [ ]:
# FORMAT
# [(Start of the entity to train, End of the entity, Type of entity)]
TRAIN_DATA = [
    ('Con Natalia Montoya de Sura', {
        'entities': [(23, 27, 'ORG')]
    }),
    ('Con Natalia Montoya de Suramericana', {
        'entities': [(23, 35, 'ORG')]
    }),
    ('Estoy validando contigo si ya te llego la documentación', {
    'entities': [(42, 55, 'PRODUCT')]
    }),
    ('confirmar que la evidencia de tu póliza quedó desde el 23 de marzo del 2024', {
        'entities': [(55, 75, 'DATE'), (33, 39, 'PRODUCT')]
    }),
    ('Sí, claro. Por PSD de la página de Seguro Sura.', {
        'entities': [(35, 47, 'PRODUCT')]
    }),
    ('Exacto, pero si llegas a tener algún tipo de accidente, la aseguradora te va. Nosotros te prestamos el servicio.', {
        'entities': [(59, 71, 'PRODUCT')]
    }),
    # ('confirmar que la evidencia de tu póliza quedó desde el 23 de marzo del 2024', {
    #     'entities': [(55, 75, 'DATE'), (33, 39, 'MISC')]
    # }),
    # ('confirmar que la evidencia de tu póliza quedó desde el 23 de marzo del 2024', {
    #     'entities': [(55, 75, 'DATE'), (33, 39, 'MISC')]
    # }),
]

variables required for the training model to be processed.

In [ ]:
# OS BASED PATH
# base = Path(__file__).resolve().parent
# Notebook Path
base = os.path.abspath('')

output_path = os.path.join(base, "likers_words")

model = None
output_dir=Path(output_path)
n_iter=100

In [ ]:
#load the model

if model is not None:
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('es')
    print("Created blank 'es' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'es' model


 train the recognizer by disabling the unnecessary pipeline except for NER. The nlp_update function can be used to train the recognizer.

In [ ]:
from spacy.training.example import Example

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


# for batch in spacy.util.minibatch(TRAIN_DATA, size=2):
#     for text, annotations in batch:
#         # create Example
#         doc = nlp.make_doc(text)
#         example = Example.from_dict(doc, annotations)
#         # Update the model
#         nlp.update([example], drop=0.3)

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # for text, annotations in tqdm(TRAIN_DATA):

        for batch in spacy.util.minibatch(tqdm(TRAIN_DATA), size=2):
            for text, annotations in batch:
                # create Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], drop=0.3)
                nlp.update(
                  [example],
                  drop=0.5,
                  sgd=optimizer,
                  losses=losses)

            # nlp.update(
            #     [text],
            #     [annotations],
            #     drop=0.5,
            #     sgd=optimizer,
            #     losses=losses)
        print(losses)

## Test trained model

In [ ]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('documentación', 'PRODUCT')]
Entities [('póliza', 'PRODUCT'), ('23 de marzo del 2024', 'DATE')]
Entities [('Suramericana', 'ORG')]
Entities [('Seguro Sura.', 'PRODUCT')]
Entities []
Entities [('Sura', 'ORG')]


Save trained model

In [ ]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/likers_words


Export model folder

In [ ]:
!zip -r /content/likers_words.zip /content/likers_words

Load trained model

In [ ]:
import spacy
from spacy import displacy
import spacy

def trained_speaker_data(speaker_data):
    """
    Process speaker data using spaCy.

    Args:
    - speaker_data: Dictionary containing the speaker's lines as values.

    Returns:
    - entities: List of entity labels found in the speaker's lines.
    """
    result_dict = {}

    print("Loading from", output_dir)
    trained_nlp = spacy.load(output_dir)

    for key, value in speaker_data.items():
        lines = [trained_nlp(line) for line in speaker_data[key]]
        result_dict[key] = [ent.label_ for line in lines for ent in line.ents]

    return result_dict

speaker1_entities = process_speaker_data(speakers1_dict)
speaker2_entities = process_speaker_data(speakers2_dict)

# speaker1_entities = result_spk1['r']
# speaker2_entities = result_spk2['r']

print(speaker1_entities)
print(speaker2_entities)

{'4339_6': ['MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'LOC', 'LOC', 'LOC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC'], '8701_2': ['PER', 'PER', 'PER', 'PER', 'MISC', 'LOC', 'MISC', 'MISC', 'MISC', 'MISC', 'PER'], '9230_3': ['PER', 'LOC', 'MISC', 'PER', 'MISC', 'MISC', 'PER', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'LOC', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'LOC', 'ORG', 'MISC', 'LOC', 'MISC', 'MISC', 'ORG', 'LOC', 'MISC', 'ORG', 'LOC', 'ORG', 'LOC', 'LOC', 'MISC', 'MISC', 'LOC', 'MISC', 'MISC', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'PER', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'LOC', 'PER', 'MISC', 'MISC', 'MISC', 'LOC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'ORG', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'ORG', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'MISC', 'PER', 

## Testing trained model

In [ ]:
displacy.render(trained_nlp(speakers1_dict[0][0]), style="ent", jupyter=True)

In [ ]:
# print("Loading from", output_dir)
# trained_nlp = spacy.load(output_dir)

def count_entity_labels(speakers_dict, model_output_dir, speaker):
    # Load the trained NLP model
    print("Loading from", model_output_dir)
    trained_nlp = spacy.load(model_output_dir)

    # Initialize a dictionary to store the counts for each speaker
    speaker_entity_counts = {}

    # Iterate over each speaker's lines
    for key, lines in speakers_dict.items():
        # Initialize a dictionary to store the counts of entity labels for this speaker
        entity_counts = {}

        # Iterate over each line for the current speaker
        for line in lines:
            # Process the line with the trained NLP model
            doc = trained_nlp(line)

            # Count the occurrences of each entity label in this line
            for ent in doc.ents:
                entity_label = ent.label_
                entity_counts[entity_label] = entity_counts.get(entity_label, 0) + 1

        # Add the counts for this speaker to the main dictionary
        speaker_entity_counts[f'{speaker}_{key}'] = entity_counts

    return speaker_entity_counts

entity_counts_spk1 = count_entity_labels(speakers1_dict, output_dir, "speaker1")
entity_counts_spk2 = count_entity_labels(speakers2_dict, output_dir, "speaker2")
print(entity_counts_spk1)
print(entity_counts_spk2)

# displacy.render(trained_nlp(line), style="ent", jupyter=True)

Loading from /content/likers_words
Loading from /content/likers_words
{'speaker1_4339_6': {'PRODUCT': 2}, 'speaker1_8701_2': {'ORG': 2, 'PRODUCT': 3, 'DATE': 1}, 'speaker1_9230_3': {'ORG': 2, 'DATE': 8, 'PRODUCT': 10}, 'speaker1_4369_1': {'PRODUCT': 17, 'DATE': 8, 'ORG': 1}}
{'speaker2_4339_6': {'PRODUCT': 7, 'DATE': 1, 'ORG': 2}, 'speaker2_8701_2': {}, 'speaker2_9230_3': {'DATE': 1, 'PRODUCT': 1}, 'speaker2_4369_1': {'PRODUCT': 2, 'ORG': 2, 'DATE': 7}}


In [ ]:
def find_seller_for_each_audio(speaker_dict1:dict, speaker_dict2:dict) -> dict:
    # Comparing the labels in speaker_dict1 to speaker_dict2
    seller_in_audio = {}

    for speaker1, entities1 in speaker_dict1.items():

        if len(speaker1.split('_')) > 2:
          index = '_'.join(speaker1.split('_')[1:])
        else:
          index = speaker1.split('_')[1]

        entity_comparission = {'speaker1':0, 'speaker2':0}

        for entity1, count1 in entities1.items():
            if entity1 in speaker_dict2[f'speaker2_{index}']:
                if count1 > speaker_dict2[f'speaker2_{index}'][entity1]:
                    entity_comparission['speaker1'] += 1
                elif count1 < speaker_dict2[f'speaker2_{index}'][entity1]:
                    entity_comparission['speaker2'] += 1
                else:
                    entity_comparission['speaker1'] += 1
                    entity_comparission['speaker2'] += 1
            else:
                entity_comparission['speaker1'] += 1

        for entity2, count2 in speaker_dict2[f'speaker2_{index}'].items():
            if entity2 not in entities1:
                entity_comparission['speaker2'] += 1
        # Save the index of the audio and the seller, which is the one that has the greater number in entities
        seller_in_audio[index] = 'speaker1' if entity_comparission['speaker1'] > entity_comparission['speaker2'] else 'speaker2'

    return seller_in_audio

seller_for_each_audio = find_seller_for_each_audio(entity_counts_spk1, entity_counts_spk2)
print(seller_for_each_audio)


{'4339_6': 'speaker2', '8701_2': 'speaker1', '9230_3': 'speaker1', '4369_1': 'speaker1'}


In [ ]:
print(speakers1_times)
print(speakers2_times)

{'4339_6': ['0:00:11 - 0:00:13', '0:00:16 - 0:00:18', '0:00:20 - 0:00:31', '0:00:32 - 0:00:36', '0:00:37 - 0:00:40', '0:00:43 - 0:00:46', '0:00:52 - 0:00:57', '0:00:57 - 0:01:09', '0:01:10 - 0:01:15', '0:01:18 - 0:01:23', '0:01:25 - 0:01:45', '0:01:46 - 0:02:28', '0:02:31 - 0:02:57', '0:02:59 - 0:03:12', '0:03:15 - 0:04:03', '0:04:06 - 0:04:21', '0:04:25 - 0:04:31', '0:04:37 - 0:04:39', '0:04:40 - 0:04:42', '0:04:45 - 0:04:47', '0:04:57 - 0:05:05', '0:05:14 - 0:05:23', '0:05:24 - 0:05:29', '0:05:34 - 0:05:35', '0:05:36 - 0:05:37', '0:05:44 - 0:05:46', '0:05:47 - 0:05:53', '0:05:59 - 0:06:15', '0:06:19 - 0:06:28', '0:06:40 - 0:07:23', '0:07:28 - 0:07:33', '0:07:40 - 0:07:42', '0:07:52 - 0:07:53', '0:07:59 - 0:08:06', '0:08:17 - 0:08:18', '0:08:19 - 0:08:25', '0:08:26 - 0:08:30', '0:08:36 - 0:08:46', '0:08:52 - 0:09:01', '0:09:06 - 0:09:09', '0:09:14 - 0:09:43', '0:09:44 - 0:09:50', '0:09:53 - 0:09:56'], '8701_2': ['0:00:11 - 0:00:13', '0:00:16 - 0:00:18', '0:00:20 - 0:00:31', '0:00:32 -

In [ ]:
from IPython.display import Audio
from scipy.io import wavfile # to read and write audio files
import re
import os
from glob import glob

# Function to convert time string to seconds
def time_to_seconds(time_str):
    parts = list(map(int, re.split('[:]', time_str)))
    return parts[0] * 3600 + parts[1] * 60 + parts[2]

def PlayAudioSegment(filepath, start, end, channel='none'):

    # get sample rate and audio data
    sample_rate, audio_data = wavfile.read(filepath) # where filepath = 'directory/audio.wav'

    #get length in minutes of audio file
    print('duration: ', audio_data.shape[0] / sample_rate / 60,'min')

    ## splice the audio with prefered start and end times
    spliced_audio = audio_data[start * sample_rate : end * sample_rate, :]

    ## choose left or right channel if preferred (0 or 1 for left and right, respectively; or leave as a string to keep as stereo)
    spliced_audio = spliced_audio[:,channel] if type(channel)==int else spliced_audio

    ## playback natively with IPython; shape needs to be (nChannel,nSamples)
    return IPython.display.Audio(spliced_audio.T, rate=sample_rate)

def play_first_interval(timestamp:dict, key):
  first_interval = timestamp[key][0]
  start, end = map(time_to_seconds, re.split('[-]', first_interval))
  duration = end - start
  print(f"Playing the first interval of key '{key}'...")


  # Get audio file with dict key

  base = os.path.abspath('')
  file_paths_wav = glob(f"{base}/*.WAV")
  input_files_wav = [os.path.basename(file_path) for file_path in file_paths_wav]
  for wav in input_files_wav:
    command = f"ffmpeg -y -i {wav} -acodec {codec} -ar 44100 dummy.wav"
    os.system(command)
    if wav.endswith(f"{key}.WAV"):
      PlayAudioSegment("dummy.wav", start, end, channel='none')


play_first_interval(speakers2_times,'4339_6')
play_first_interval(speakers1_times,'8701_2')
play_first_interval(speakers1_times,'9230_3')
play_first_interval(speakers1_times,'4369_1')
